# LOADING DATASET

In [1]:
import math
import pathlib
import random
import urllib

from torchinfo import summary

import time
import wandb
import zstandard
import chess
import torch
import numpy as np
from torch import nn, Tensor
from torch.utils.data import DataLoader

In [2]:
wandb.login()

wandb: Currently logged in as: spajro (spajro-chess). Use `wandb login --relogin` to force relogin


True

In [3]:
def __download(url: str, name: str) -> str:
    path, _ = urllib.request.urlretrieve(url, name)
    return path


def __unpack(path: str, name: str):
    input_file = pathlib.Path(path)
    with open(input_file, 'rb') as compressed:
        decomp = zstandard.ZstdDecompressor()
        output_path = name
        with open(output_path, 'wb') as destination:
            decomp.copy_stream(compressed, destination)
            destination.close()
        compressed.close()


def __remove(path: str):
    pathlib.Path.unlink(pathlib.Path(path))

In [4]:
path = __download("https://database.lichess.org/lichess_db_puzzle.csv.zst", "lichess_db_puzzle.csv.zst")

In [5]:
__unpack(path, "lichess_db_puzzle.csv")

In [6]:
__remove("lichess_db_puzzle.csv.zst")

In [7]:
class Puzzle:
    def __init__(self, row: str):
        fields = row.split(',')
        self.fen = fields[1]
        self.moves = fields[2].split(" ")
        self.tags = fields[7].split(" ")
        self.board = chess.Board(self.fen)

    def __str__(self):
        return "{fen: " + self.fen + " ,tags: [" + ", ".join(self.tags) + "],moves: [" + ",".join(self.moves) + "]}"

    def fen_to_tensors_list(self) -> [torch.Tensor]:
        white = self.board.occupied_co[chess.WHITE]
        black = self.board.occupied_co[chess.BLACK]

        return [
            bitboard_to_tensor(white),
            bitboard_to_tensor(black),
            bitboard_to_tensor(white & self.board.kings),
            bitboard_to_tensor(black & self.board.kings),
            bitboard_to_tensor(white & self.board.pawns),
            bitboard_to_tensor(black & self.board.pawns),
            bitboard_to_tensor(white & self.board.queens),
            bitboard_to_tensor(black & self.board.queens),
            bitboard_to_tensor(white & self.board.knights),
            bitboard_to_tensor(black & self.board.knights),
            bitboard_to_tensor(white & self.board.bishops),
            bitboard_to_tensor(black & self.board.bishops),
            bitboard_to_tensor(white & self.board.rooks),
            bitboard_to_tensor(black & self.board.rooks)
        ]

    def puzzle_to_tensor(self) -> torch.Tensor:
        fen_tensors = self.fen_to_tensors_list()
        move_tensors: list[Tensor] = []
        for i in range(7):  # FIRST 7 moves
            if i < len(self.moves):
                move_tensors.append(move_to_tensor(self.moves[i]))
            else:
                move_tensors.append(torch.zeros(8, 8))
        return torch.stack(fen_tensors + move_tensors)

In [8]:
def load(k: int) -> [Puzzle]:
    f = open("lichess_db_puzzle.csv")
    f.readline()
    result = []
    for i in range(k):
        result.append(Puzzle(f.readline()))
    f.close()
    return result

In [9]:
load(10)[0].__str__()

'{fen: r6k/pp2r2p/4Rp1Q/3p4/8/1N1P2R1/PqP2bPP/7K b - - 0 24 ,tags: [crushing, hangingPiece, long, middlegame],moves: [f2g3,e6e7,b2b1,b3c1,b1c1,h6c1]}'

# FILTER DATASET

In [10]:
expected_tags = {
    'attraction',
    'discoveredAttack',
    'doubleCheck',
    'fork',
    'pin',
    'sacrifice',
    'skewer',
    'xRayAttack',
    'zugzwang',
    'deflection',
    'clearance'
}

In [11]:
expected_tags_list = list(expected_tags)

In [12]:
def not_has_promotion_move(puzzle: Puzzle) -> bool:
    return all([len(m) == 4 for m in puzzle.moves])


def filter_data(data: [Puzzle]) -> [Puzzle]:
    puzzles_single_tag = list(filter(lambda p: len(set(p.tags) & expected_tags) == 1, data))
    puzzles_not_ambiguous = list(filter(not_has_promotion_move, puzzles_single_tag))
    return puzzles_not_ambiguous


In [13]:
len(filter_data(load(100)))

35

# CONVERSION TO TENSOR

In [14]:
def bitboard_to_tensor(bitboard: int) -> torch.Tensor:
    li = [1 if digit == '1' else 0 for digit in bin(bitboard)[2:]]
    li = [0 for _ in range(64 - len(li))] + li
    return torch.tensor(li).reshape((8, 8))

In [15]:
load(1)[0].fen_to_tensors_list()

[tensor([[0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [1, 0, 0, 1, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 1, 0, 0, 1, 0, 1, 0],
         [1, 1, 0, 0, 0, 1, 0, 1],
         [1, 0, 0, 0, 0, 0, 0, 0]]),
 tensor([[1, 0, 0, 0, 0, 0, 0, 1],
         [1, 0, 0, 1, 0, 0, 1, 1],
         [0, 0, 1, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 1, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 1, 0, 0, 0, 1, 0],
         [0, 0, 0, 0, 0, 0, 0, 0]]),
 tensor([[0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [1, 0, 0, 0, 0, 0, 0, 0]]),
 tensor([[1, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0

In [16]:
def move_to_tensor(move: str) -> torch.Tensor:
    x1 = 7 - ord(move[0]) + ord('a')
    y1 = 8 - int(move[1])
    x2 = 7 - ord(move[2]) + ord('a')
    y2 = 8 - int(move[3])
    tensor = torch.zeros(8, 8)
    tensor[y1][x1] = 1
    tensor[y2][x2] = 1
    return tensor

In [17]:
print(move_to_tensor('b2b4'))

tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])


In [18]:
load(1)[0].puzzle_to_tensor()

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 1., 0.,  ..., 0., 1., 0.],
         [1., 1., 0.,  ..., 1., 0., 1.],
         [1., 0., 0.,  ..., 0., 0., 0.]],

        [[1., 0., 0.,  ..., 0., 0., 1.],
         [1., 0., 0.,  ..., 0., 1., 1.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 1., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0., 

# CONVERT AND BATCH DATASET

In [19]:
def puzzle_to_tag_index(puzzle: Puzzle) -> int:
    [tag] = set(puzzle.tags) & expected_tags
    index = expected_tags_list.index(tag)
    return index

In [20]:
puzzle_to_tag_index(filter_data(load(100))[0])

3

In [21]:
def convert_dataset(puzzles: [Puzzle]) -> list[tuple[torch.Tensor, int]]:
    return [(puzzle.puzzle_to_tensor(), puzzle_to_tag_index(puzzle)) for puzzle in puzzles]

In [22]:
TEST_SIZE = 500 * 64
VALIDATION_SIZE = 500 * 64

In [23]:
dataset = convert_dataset(filter_data(load(3000000)))
random.shuffle(dataset)
len(dataset)

886146

In [24]:
dataset_test, dataset = dataset[:TEST_SIZE], dataset[TEST_SIZE:]
dataset_valid, dataset_train = dataset[:VALIDATION_SIZE], dataset[VALIDATION_SIZE:]

print("Train: ", len(dataset_train))
print("Test: ", len(dataset_test))
print("Validation: ", len(dataset_valid))

Train:  822146
Test:  32000
Validation:  32000


In [25]:
BATCH_SIZE = 64

In [26]:
loader_train = DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True)
loader_test = DataLoader(dataset_test, batch_size=BATCH_SIZE, shuffle=True)
loader_valid = DataLoader(dataset_valid, batch_size=BATCH_SIZE, shuffle=True)

print("Batches train", len(loader_train))
print("Batches valid", len(loader_valid))
print("Batches test", len(loader_test))

Batches train 12847
Batches valid 500
Batches test 500


# TRAINING

In [27]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DATA_SHAPE = (BATCH_SIZE,) + tuple(dataset_train[0][0].shape)

In [28]:
class Model(nn.Module):
    def __init__(self, *args, **kwargs):
        super(Model, self).__init__()
        self.classifier = nn.Sequential(*args, **kwargs)

    def init_xavier(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight)

    def calc_accuracy(self, loader):
        correct = 0
        total = 0
        self.eval()
        with torch.no_grad():
            for batch in loader:
                features, labels = batch
                labels = labels.to(DEVICE)
                features = features.to(DEVICE)
                outputs = self(features)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        self.train()
        return correct / total

    def forward(self, X):
        return self.classifier.forward(X)

In [29]:
def create_model0():
    return Model(nn.Conv2d(21, 8 * 8, kernel_size=4, padding=4),
                 nn.ReLU(),
                 nn.Conv2d(8 * 8, 4 * 4, kernel_size=2, padding=4),
                 nn.ReLU(),
                 nn.MaxPool2d(kernel_size=4, stride=1),
                 nn.Conv2d(4 * 4, 8 * 8, kernel_size=2, padding=4),
                 nn.ReLU(),
                 nn.Conv2d(8 * 8, 1, kernel_size=4, padding=4),
                 nn.ReLU(),
                 nn.Flatten(),
                 nn.Linear(841, 256),
                 nn.ReLU(),
                 nn.Linear(256, 64),
                 nn.ReLU(),
                 nn.Linear(64, 11)
                 )


model0 = create_model0()
architecture_summary = summary(model0, DATA_SHAPE)
print(architecture_summary)

D:\Projekty\TacticsClassifier\.venv\Lib\site-packages\torch\nn\modules\conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ..\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


Layer (type:depth-idx)                   Output Shape              Param #
Model                                    [64, 11]                  --
├─Sequential: 1-1                        --                        --
│    └─Conv2d: 2-1                       [64, 64, 13, 13]          21,568
│    └─ReLU: 2-2                         [64, 64, 13, 13]          --
│    └─Conv2d: 2-3                       [64, 16, 20, 20]          4,112
│    └─ReLU: 2-4                         [64, 16, 20, 20]          --
│    └─MaxPool2d: 2-5                    [64, 16, 17, 17]          --
│    └─Conv2d: 2-6                       [64, 64, 24, 24]          4,160
│    └─ReLU: 2-7                         [64, 64, 24, 24]          --
│    └─Conv2d: 2-8                       [64, 1, 29, 29]           1,025
│    └─ReLU: 2-9                         [64, 1, 29, 29]           --
│    └─Flatten: 2-10                     [64, 841]                 --
│    └─Linear: 2-11                      [64, 256]                 215,5

In [30]:
def create_model1():
    return Model(nn.Conv2d(21, 300, kernel_size=5, padding=5),
                 nn.ReLU(),
                 nn.Conv2d(300, 30, kernel_size=2, padding=2),
                 nn.ReLU(),
                 nn.Flatten(),
                 nn.Dropout(0.5),
                 nn.Linear(8670, 20),
                 nn.ReLU(),
                 nn.Linear(20, 11)
                 )


model1 = create_model1()
architecture_summary = summary(model1, DATA_SHAPE)
print(architecture_summary)

Layer (type:depth-idx)                   Output Shape              Param #
Model                                    [64, 11]                  --
├─Sequential: 1-1                        --                        --
│    └─Conv2d: 2-1                       [64, 300, 14, 14]         157,800
│    └─ReLU: 2-2                         [64, 300, 14, 14]         --
│    └─Conv2d: 2-3                       [64, 30, 17, 17]          36,030
│    └─ReLU: 2-4                         [64, 30, 17, 17]          --
│    └─Flatten: 2-5                      [64, 8670]                --
│    └─Dropout: 2-6                      [64, 8670]                --
│    └─Linear: 2-7                       [64, 20]                  173,420
│    └─ReLU: 2-8                         [64, 20]                  --
│    └─Linear: 2-9                       [64, 11]                  231
Total params: 367,481
Trainable params: 367,481
Non-trainable params: 0
Total mult-adds (Units.GIGABYTES): 2.66
Input size (MB): 0.34
Forwar

In [31]:
def create_model2():
    return Model(nn.Conv2d(21, 300, kernel_size=5, padding=5),
                 nn.ReLU(),
                 nn.BatchNorm2d(300),
                 nn.Conv2d(300, 30, kernel_size=2, padding=2),
                 nn.ReLU(),
                 nn.BatchNorm2d(30),
                 nn.Conv2d(30, 30, kernel_size=2, padding=2),
                 nn.ReLU(),
                 nn.Flatten(),
                 nn.Dropout(0.5),
                 nn.Linear(12000, 30),
                 nn.ReLU(),
                 nn.Linear(30, 11)
                 )


model2 = create_model2()
architecture_summary = summary(model2, DATA_SHAPE)
print(architecture_summary)

Layer (type:depth-idx)                   Output Shape              Param #
Model                                    [64, 11]                  --
├─Sequential: 1-1                        --                        --
│    └─Conv2d: 2-1                       [64, 300, 14, 14]         157,800
│    └─ReLU: 2-2                         [64, 300, 14, 14]         --
│    └─BatchNorm2d: 2-3                  [64, 300, 14, 14]         600
│    └─Conv2d: 2-4                       [64, 30, 17, 17]          36,030
│    └─ReLU: 2-5                         [64, 30, 17, 17]          --
│    └─BatchNorm2d: 2-6                  [64, 30, 17, 17]          60
│    └─Conv2d: 2-7                       [64, 30, 20, 20]          3,630
│    └─ReLU: 2-8                         [64, 30, 20, 20]          --
│    └─Flatten: 2-9                      [64, 12000]               --
│    └─Dropout: 2-10                     [64, 12000]               --
│    └─Linear: 2-11                      [64, 30]                  360,0

In [32]:
CREATE_MODELS_LIST = [create_model0, create_model1, create_model2]


def create_model(model_number: int):
    return CREATE_MODELS_LIST[model_number]()

In [33]:
def accuracy(out, truth):
    return torch.argmax(out, dim=1) == truth


def train(model, criterion, optimizer, epoch):
    model = model.to(DEVICE)
    model.init_xavier()
    criterion = criterion.to(DEVICE)
    batches_amount = len(loader_train)

    for i in range(epoch):
        time_started = time.time() * 1000
        loss_sum = 0.0
        accuracy_sum = 0.0
        for batch, truth in loader_train:
            batch = batch.to(DEVICE)
            truth = truth.to(DEVICE)

            optimizer.zero_grad()
            out = model.forward(batch)
            loss = criterion(out, truth)
            loss.backward()
            optimizer.step()

            accuracy_value = accuracy(out, truth).sum() / BATCH_SIZE

            loss_sum += loss.item()
            accuracy_sum += accuracy_value.item()

        passed_time = math.ceil(time.time() * 1000 - time_started)
        loss_average = loss_sum / batches_amount
        accuracy_average = accuracy_sum / batches_amount
        validation_accuracy = model.calc_accuracy(loader_valid)

        print(
            f"Epoch [{i + 1}/{epoch}], train_loss: {loss_average}, train_accuracy: {accuracy_average}, validation_accuracy: {validation_accuracy}, time: {passed_time / 1000}s")
        wandb.log({
            "epoch": i + 1,
            "train_loss": loss_average,
            "train_accuracy": accuracy_average,
            "validation_accuracy": validation_accuracy}
        )

In [34]:
### MAIN SWEEP CONFIG
sweep_config = {
    'method': 'bayes',  # random, grid or bayes
    'name': 'sweep-chess-tactics',
    'metric': {
        'goal': 'maximize',
        'name': 'validation_accuracy'
    },
    'parameters': {
        'epochs': {
            'values': [20, 30, 50]
        },
        'learning_rate': {
            'values': [0.0003, 0.001, 0.01, 0.1],
        },
        'optimizer': {
            'values': ['adam','sgd']
        },
        'criterion': {
            'values': ['CrossEntropy']
        },
        'model': {
            'values': list(range(len(CREATE_MODELS_LIST)))
        }
    },

}

In [35]:
sweep_id = wandb.sweep(sweep_config, project="raport")

Create sweep with ID: ai82s9v2
Sweep URL: https://wandb.ai/spajro-chess/raport/sweeps/ai82s9v2


In [36]:
def create_optimizer(model, optimizer="adam"):
    if optimizer == "sgd":
        optimizer = torch.optim.SGD
    elif optimizer == "adam":
        optimizer = torch.optim.Adam
    return optimizer


def create_criterion(criterion="CrossEntropy"):
    if criterion == "CrossEntropy":
        return nn.CrossEntropyLoss()
    if criterion == 'Nll':
        return nn.NLLLoss()


def main():
    with wandb.init(config=None) as run:
        config = wandb.config
        run.name = f"model{config['model']}_epochs{config['epochs']}_learning_rate{config['learning_rate']}_optimizer{config['optimizer']}"

        model = create_model(int(config['model']))
        opt_fn = create_optimizer(model, wandb.config.optimizer)
        crt_fn = create_criterion(wandb.config.criterion)
        train(model, crt_fn, opt_fn(model.classifier.parameters(), lr=wandb.config.learning_rate), wandb.config.epochs)
        model_path = f"{run.name}.save"
        torch.save(model, model_path)


wandb.agent(sweep_id, main)

wandb: Agent Starting Run: 56uawfua with config:
wandb: 	criterion: CrossEntropy
wandb: 	epochs: 20
wandb: 	learning_rate: 0.1
wandb: 	model: 2
wandb: 	optimizer: sgd


D:\Projekty\TacticsClassifier\.venv\Lib\site-packages\torch\nn\modules\conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ..\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


Epoch [1/20], train_loss: 0.9226569359178366, train_accuracy: 0.6976482836459874, validation_accuracy: 0.1810625, time: 27.888s
Epoch [2/20], train_loss: 0.6257731031202811, train_accuracy: 0.7962814762201292, validation_accuracy: 0.08640625, time: 27.316s
Epoch [3/20], train_loss: 0.5506008906452176, train_accuracy: 0.8202936483225656, validation_accuracy: 0.836625, time: 26.185s
Epoch [4/20], train_loss: 0.5061155076461923, train_accuracy: 0.8345820035806025, validation_accuracy: 0.7145, time: 26.097s
Epoch [5/20], train_loss: 0.474216917147662, train_accuracy: 0.8449601560675644, validation_accuracy: 0.8286875, time: 26.96s
Epoch [6/20], train_loss: 0.4511170310157573, train_accuracy: 0.8519681151241535, validation_accuracy: 0.85515625, time: 29.199s
Epoch [7/20], train_loss: 0.4316834721488369, train_accuracy: 0.8583606581303028, validation_accuracy: 0.82840625, time: 26.192s
Epoch [8/20], train_loss: 0.4160207412296616, train_accuracy: 0.8635527749669184, validation_accuracy: 0.66

epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
train_accuracy,▁▅▅▆▆▇▇▇▇▇▇▇▇███████
train_loss,█▅▄▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁
validation_accuracy,▂▁█▇███▆██▅▃▇█████▂█
epoch,20
train_accuracy,0.89235
train_loss,0.32631
validation_accuracy,0.87606


wandb: Agent Starting Run: 5phy00yy with config:
wandb: 	criterion: CrossEntropy
wandb: 	epochs: 30
wandb: 	learning_rate: 0.1
wandb: 	model: 1
wandb: 	optimizer: sgd


...

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train_accuracy,▁███████████████████████████████████████
train_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
validation_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,50
train_accuracy,0.38369
train_loss,1.86727
validation_accuracy,0.38334


wandb: Agent Starting Run: b8maocqt with config:
wandb: 	criterion: CrossEntropy
wandb: 	epochs: 30
wandb: 	learning_rate: 0.0003
wandb: 	model: 2
wandb: 	optimizer: sgd


Epoch [1/30], train_loss: 1.7011769226555546, train_accuracy: 0.41516623530785396, validation_accuracy: 0.47825, time: 28.525s
Epoch [2/30], train_loss: 1.4413240261371845, train_accuracy: 0.5076647271736592, validation_accuracy: 0.56178125, time: 27.837s
Epoch [3/30], train_loss: 1.2769519493553325, train_accuracy: 0.5725570658519499, validation_accuracy: 0.611, time: 27.94s
Epoch [4/30], train_loss: 1.1708477261169064, train_accuracy: 0.6123134292052619, validation_accuracy: 0.6435625, time: 27.868s
Epoch [5/30], train_loss: 1.0881577348016882, train_accuracy: 0.6414824472639526, validation_accuracy: 0.6643125, time: 29.195s
Epoch [6/30], train_loss: 1.0164149535707234, train_accuracy: 0.6665709893360318, validation_accuracy: 0.69134375, time: 28.009s
Epoch [7/30], train_loss: 0.9569363341221431, train_accuracy: 0.6861767338678291, validation_accuracy: 0.7095, time: 28.433s
Epoch [8/30], train_loss: 0.9065641161253585, train_accuracy: 0.7028270218728108, validation_accuracy: 0.719375

epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
train_accuracy,▁▃▄▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇██████████
train_loss,█▆▅▅▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
validation_accuracy,▁▃▄▄▅▅▆▆▆▇▇▇▇▇▇▇▇▇▇▇█▇▇████▃██
epoch,30
train_accuracy,0.82003
train_loss,0.54352
validation_accuracy,0.80541


wandb: Ctrl + C detected. Stopping sweep.


In [37]:
wandb.finish()